In [1]:
import json, glob, boto3, os
import pdb
import pandas as pd
from pandas.io.json import json_normalize

# Testing out json processing

In [2]:
# from https://alexwlchan.net/2019/07/listing-s3-keys/
def get_matching_s3_objects(bucket, prefix="", suffix=""):
    """
    Generate objects in an S3 bucket.

    :param bucket: Name of the S3 bucket.
    :param prefix: Only fetch objects whose key starts with
        this prefix (optional).
    :param suffix: Only fetch objects whose keys end with
        this suffix (optional).
    """
    s3 = boto3.client("s3")
    paginator = s3.get_paginator("list_objects_v2")

    kwargs = {'Bucket': bucket}

    # We can pass the prefix directly to the S3 API.  If the user has passed
    # a tuple or list of prefixes, we go through them one by one.
    if isinstance(prefix, str):
        prefixes = (prefix, )
    else:
        prefixes = prefix

    for key_prefix in prefixes:
        kwargs["Prefix"] = key_prefix

        for page in paginator.paginate(**kwargs):
            try:
                contents = page["Contents"]
            except KeyError:
                return

            for obj in contents:
                key = obj["Key"]
                if key.endswith(suffix):
                    yield obj


def get_matching_s3_keys(bucket, prefix="", suffix=""):
    """
    Generate the keys in an S3 bucket.

    :param bucket: Name of the S3 bucket.
    :param prefix: Only fetch keys that start with this prefix (optional).
    :param suffix: Only fetch keys that end with this suffix (optional).
    """
    for obj in get_matching_s3_objects(bucket, prefix, suffix):
        yield obj["Key"]

In [3]:
session = boto3.Session()
BUCKET_NAME = 'snowbot-pv'

# S3 Connect
s3 = session.resource('s3')

bucket = s3.Bucket(BUCKET_NAME)

In [10]:
DATA_DIR = "../data/"
MERGED_FILENAME = "merged_file.json"
merged_file = DATA_DIR + MERGED_FILENAME

TEST_FILENAME = "test_file.json"
merged_test_file = DATA_DIR + TEST_FILENAME


In [5]:
def merge_matching_jsons(save_file, suffix=""):
    
    result = []

    for f in get_matching_s3_keys(BUCKET_NAME, suffix=suffix):

        # Write the file from S3 into a local temp file
        with open('temp', 'wb') as tfw:
            bucket.download_fileobj(f, tfw)

        # Append the local temp file into the result list
        with open('temp', 'rb') as tfr:          
            result.append(json.load(tfr))

    os.remove("temp")

    # Fill the output file with the merged content
    with open(save_file, "w") as outfile:
         json.dump(result, outfile)
            
# TBD: more efficient to go straight to df w/o saving json to file

In [6]:
def load_merged_json_as_df(merged_file):
    #load the merged json as a dataframe
    with open(merged_file, "r") as f:
        d = json.load(f)
        df = pd.DataFrame.from_dict(json_normalize(d, record_path='lifts', meta='timestamp'))
        return df

In [7]:
def set_lifts_df_datatypes(df):

    # set datatypes for lift table
    df = df.astype({
        "liftID": 'category',
        "resortID": 'category',
        "liftName": 'category',
        "status": 'category',
        "timeToRide": "int"
    })
    df["timestamp"] = pd.to_datetime(df["timestamp"])
    
    return df

In [8]:
def get_status_changes(df):
    '''Returns a dataframe that only includes the times when there was a change to a new status'''
    
    def calc_status_change(df):
        firstrow = df.loc[df['timestamp'].idxmin()]
        change_rows = df[df.status.ne(df.status.shift())]
        keep_df = firstrow.to_frame().T.append(change_rows)
        
        # Remove so that we don't need to write another column to S3 as we scrape?
        # Just calculate it when plotting and predicting?
        # keep_df['time_diff'] = keep_df['timestamp'].diff(1).shift(-1)
        
        return keep_df
        
    
    df = df.groupby('liftName', group_keys=False)\
           .apply(calc_status_change)\
           .reset_index(drop=True)
    
    return df

# TBD: may need to convert timestamp to days (e.g. for Tableau)

# Process lift json fies

In [11]:
merge_matching_jsons(suffix="lifts.json", save_file=merged_file)
lifts_df = load_merged_json_as_df(merged_file)
lifts_df = set_lifts_df_datatypes(lifts_df)

lifts_status_changes_df = get_status_changes(lifts_df)

**NOTE:** `timeToRide` is just the time is takes to ride the lift, not the current wait time:

In [12]:
lifts_df.groupby("liftName")['timeToRide'].unique()

liftName
7th Heaven Express                 [6]
Big Red Express                    [8]
Blackcomb Gondola Lower            [7]
Blackcomb Gondola Upper            [7]
Catskinner Express                 [4]
Coca-Cola Tube Park                [4]
Creekside Gondola                  [7]
Crystal Ridge Express              [7]
Emerald 6 Express                  [6]
Excalibur Gondola Lower            [3]
Excalibur Gondola Upper            [5]
Excelerator Express                [6]
Fitzsimmons Express                [6]
Franz's Chair                      [8]
Garbanzo Express                   [7]
Glacier Express                    [6]
Harmony 6 Express                  [6]
Horstman T-Bar                     [4]
Jersey Cream Express               [5]
Magic Chair                        [6]
Olympic Chair                      [5]
Peak 2 Peak Gondola               [12]
Peak Express                       [3]
Showcase T-Bar                     [3]
Symphony Express                   [7]
T-Bars          

In [13]:
# Get status durations
# TBD: optimize if needed via # 3 under:
# https://towardsdatascience.com/pandas-tips-and-tricks-33bcc8a40bb9
df = lifts_status_changes_df.sort_values(by=['liftID', 'timestamp'])
df['time_diff'] = df.groupby('liftID')['timestamp'].diff(1).shift(-1)

# Fill in the durations which will be missing for the most recent status changes
missing_time_diffs_idx = df.loc[(df['time_diff'].isnull()) & (
    df['timestamp'] >= df['timestamp'].min()), 'timestamp'].index.values

df.loc[missing_time_diffs_idx, 'time_diff'] = df['timestamp'].max(
) - df.loc[missing_time_diffs_idx, 'timestamp']

# Convert to seconds
df['time_diff_seconds'] = df['time_diff'].dt.total_seconds()

In [14]:
df

,liftID,resortID,liftName,status,timeToRide,timestamp,time_diff,time_diff_seconds
0,3,13,7th Heaven Express,X,6,2020-01-03 00:19:09.631011-08:00,1 days 13:59:59.794440,136799.794440
1,3,13,7th Heaven Express,O,6,2020-01-04 14:19:09.425451-08:00,0 days 00:29:59.297179,1799.297179
2,3,13,7th Heaven Express,X,6,2020-01-04 14:49:08.722630-08:00,0 days 22:00:01.500183,79201.500183
3,3,13,7th Heaven Express,O,6,2020-01-05 12:49:10.222813-08:00,0 days 01:59:58.326201,7198.326201
4,3,13,7th Heaven Express,X,6,2020-01-05 14:49:08.549014-08:00,0 days 18:56:18.469982,68178.469982
...,...,...,...,...,...,...,...,...
1827,72,13,Whistler Village Gondola Upper,H,11,2020-01-29 08:00:28.210238-08:00,0 days 00:29:58.285712,1798.285712
1828,72,13,Whistler Village Gondola Upper,O,11,2020-01-29 08:30:26.495950-08:00,0 days 07:14:59.875339,26099.875339
1829,72,13,Whistler Village Gondola Upper,X,11,2020-01-29 15:45:26.371289-08:00,0 days 15:59:59.554913,57599.554913
1830,72,13,Whistler Village Gondola Upper,H,11,2020-01-30 07:45:25.926202-08:00,0 days 00:45:00.787765,2700.787765


In [15]:
df.dtypes

liftID                                               object
resortID                                             object
liftName                                             object
status                                               object
timeToRide                                           object
timestamp            datetime64[ns, pytz.FixedOffset(-480)]
time_diff                                   timedelta64[ns]
time_diff_seconds                                   float64
dtype: object

In [16]:
# Uses locale date formatting, otherwise Tableau will mix up month and day
# alternatively, can export to json:
# lifts_status_changes_df.to_json(DATA_DIR + "lifts_status_changes.json", orient='table')
df.to_csv(DATA_DIR + "lifts_status_changes.csv", date_format='%c')


In [17]:
# add:
# 
# daily: for each chair calculate most open status of the day: O > H > X
# Days since each chair was last seen open with timestamp of most recent open time.
# snowfall since last open
# save data for other mountains

# Storage options testing

In [ ]:
df.to_pickle(DATA_DIR + "df_test.pkl")

In [ ]:
from fastparquet import write

# parquet engines don't handle shifted timezones
import pytz
TZ = pytz.timezone('America/Vancouver')
df['timestamp'] = df.timestamp.dt.tz_convert(pytz.utc)

In [ ]:
# Note: May need snappy-python as a req to run on AWS Lambda
df.to_parquet(DATA_DIR + "df_test.parquet", engine='fastparquet')

In [ ]:
load_df = pd.read_parquet(DATA_DIR + "df_test.parquet")
load_df['timestamp'] = load_df.timestamp.dt.tz_convert(TZ) # convert back to correct timezone


In [ ]:
#TBD convert back to correct datatypes
load_df.dtypes

In [ ]:
df.to_csv(DATA_DIR + "df_test.csv")

Test file size results:
- json: 800 Kb?
- csv: 474 Kb
- pickle: 145 Kb
- parquet: 15 Kb

## Delta Lake Testing

Requires apache spark instance.  For future use, could set one up to work with lambda using https://aws.amazon.com/emr/features/spark/?

Otherwise databricks (similar to QxMD project)

# json comparison and parquet to s3

In [20]:
from fastparquet import write, ParquetFile
import os
import pytz
import s3fs
import botocore

os.chdir("../src/data/snowbot_AWS_lambda/")
from scrape import get_data
os.chdir("..")

# parquet engines don't handle shifted timezones

TZ = pytz.timezone('America/Vancouver')

In [21]:
# new version

fs = s3fs.S3FileSystem()
myopen = fs.open
nop = lambda *args, **kwargs: None


fname = 'wb_lifts_history'


def write_dataframe_to_parquet_on_s3(df, fname):
    """ Write a dataframe to a Parquet on S3.  Creates a new parquet file if one doesn't already exist."""
    
    # Unshift the timezone because parquet engines don't handle shifted timezones
    df.loc[:, 'timestamp'] = df.loc[:, 'timestamp'].dt.tz_convert(pytz.utc)

    s3_object = s3.Object(BUCKET_NAME, fname + ".parquet")

    def write_parquet(df, fname, app=True):

        output_file = f"s3://{BUCKET_NAME}/{fname}.parquet"
        write(output_file,
              df,
              partition_on=['timestamp'],
              file_scheme='hive',
              append=app,  # need to remove or catch exception to work when file doesn't exist
              open_with=myopen,
              mkdirs=nop)
        print("Writing {} records to {}".format(len(df), fname))

    if not list(bucket.objects.filter(Prefix=fname)):
        print("File {} not found.  Creating file.".format(fname))
        write_parquet(df, fname, app=False)

    else:
        print("file found.  adding new stuff")
        write_parquet(df, fname, app=True)

In [22]:
def jsons_to_df(jsons):
    # repeats. functionize first occurace?
    return pd.DataFrame.from_dict(json_normalize(jsons, record_path='lifts', meta='timestamp'))

In [23]:
# Get current lift status info json
lifts_current = get_data()['lifts'] # String.  
lifts_current_json = json.loads(lifts_current)

# retrieve prior lift status info json

def save_prior(json_data):
    bucket.put_object(Key="lifts_prior.json",
                      Body=bytes(json.dumps(json_data).encode('UTF-8')))


lifts_prior_object = s3.Object(BUCKET_NAME, 'lifts_prior.json')

try:
    lifts_prior_object.load()
except botocore.exceptions.ClientError as e:
    if e.response['Error']['Code'] == "404":
        print("Prior doesn't exist")
        save_prior(lifts_current_json)  # Create the prior file
        print("Created PRIOR_FILENAME_TBD")
    else:
        # Something else has gone wrong.
        raise
else:
    # The prior exists
    file_content = lifts_prior_object.get()['Body'].read().decode('utf-8')
    json_content = json.loads(file_content)
    print("loaded prior json data from S3")

    # compare jsons without their timestamps
    if json_content['lifts'] == lifts_current_json['lifts']:
        print("No differences between current and prior data were found.")
    else:
        # Get a df with the status chages between the prior and current json data
        df = jsons_to_df([json_content, lifts_current_json])
        df = set_lifts_df_datatypes(df)  # TBD add to jsons_to_df
        df = get_status_changes(df)
        df = set_lifts_df_datatypes(df)  # TBD add to jsons_to_df
        print("Found differences")
        write_dataframe_to_parquet_on_s3(df, fname)

        # save current lift status info json as prior
        save_prior(lifts_current_json)
        print("Updated {} with current data.".format(lifts_prior_object.key))

loaded prior json data from S3
Found differences
file found.  adding new stuff
Writing 43 records to wb_lifts_history
Updated lifts_prior.json with current data.


In [24]:
# load the parquet file


def load_dataframe_from_parquet_on_s3(fname):
    """ Load a dataframe from a Parquet on S3. """
    read_file = f"s3://{BUCKET_NAME}/{fname}.parquet"
    pf = ParquetFile(read_file, open_with=myopen)
    df = pf.to_pandas()

    # Reshift the timezone because parquet engines don't handle shifted timezones
    df['timestamp'].cat.set_categories(df['timestamp'].cat.categories.tz_convert(TZ), inplace=True)
    
    return df


load_dataframe_from_parquet_on_s3(fname)

,liftID,resortID,liftName,status,timeToRide,timestamp
0,3,13,7th Heaven Express,H,6,2020-01-13 13:21:56.629410-08:00
1,36,13,Big Red Express,H,8,2020-01-13 13:21:56.629410-08:00
2,69,13,Blackcomb Gondola Lower,H,7,2020-01-13 13:21:56.629410-08:00
3,70,13,Blackcomb Gondola Upper,H,7,2020-01-13 13:21:56.629410-08:00
4,9,13,Catskinner Express,H,4,2020-01-13 13:21:56.629410-08:00
...,...,...,...,...,...,...
173,40,13,Garbanzo Express,O,7,2020-01-30 15:31:26.798102-08:00
174,6,13,Magic Chair,O,6,2020-01-30 15:31:26.798102-08:00
175,39,13,Olympic Chair,O,5,2020-01-30 15:31:26.798102-08:00
176,33,13,Whistler Village Gondola Lower,O,5,2020-01-30 15:31:26.798102-08:00


# Testing timestamps for file loading

In [ ]:
read_file = f"s3://{BUCKET_NAME}/{fname}.parquet"
pf = ParquetFile(read_file, open_with=myopen)
test = pf.to_pandas()["timestamp"]

In [ ]:
# If needed: to convert for categorical datetime to regular datetime
df["timestamp"] = pd.to_datetime(pd.Series(np.asarray(df["timestamp"])))

In [ ]:
test.dt = test.dt.tz_convert(tz= 'America/Vancouver')

/Users/paul/anaconda3/lib/python3.7/site-packages/pandas/core/series.py:597: FutureWarning: Converting timezone-aware DatetimeArray to timezone-naive ndarray with 'datetime64[ns]' dtype. In the future, this will return an ndarray with 'object' dtype where each element is a 'pandas.Timestamp' with the correct 'tz'.
	To accept the future behavior, pass 'dtype=object'.
	To keep the old behavior, pass 'dtype="datetime64[ns]"'.


more info: https://pandas-docs.github.io/pandas-docs-travis/whatsnew/v0.24.0.html#converting-timezone-aware-series-and-index-to-numpy-arrays

In [ ]:
load_dataframe_from_parquet_on_s3(fname).dtypes

### Testing local parquet saves

In [ ]:
def save_parquet(df, fname):
    # parquet engines don't handle shifted timezones
    df.loc[:, 'timestamp'] = df.loc[:, 'timestamp'].dt.tz_convert(pytz.utc)

    # Note: May need snappy-python as a req to run on AWS Lambda
    df.to_parquet(DATA_DIR + fname + '.parquet',
                  engine='fastparquet',
                  partition_on=['timestamp'],
                  file_scheme='mixed')

In [91]:
save_parquet(df[0:3].copy(), 'wb_lifts_history')

In [92]:
df.iloc[20:22, :].copy().to_parquet(DATA_DIR + 'wb_lifts_history' + '.parquet',
              engine='fastparquet',
              partition_on=['timestamp'],
              file_scheme='mixed',
              append=True)
# Catch exception that is doesn't exist here

In [ ]:
# todo: change time_diff to "duration"
# test on lambda
# make datatype dict for and general set datatypes function
